### Overview
#### Downloading Activity data for Ligand inhibitors of Target proteins

1. Download the ChEMBL database
2. Query ChEMBL for drug data
3. Remove duplicates from the ChEMBL data


Install the necessary Python libraries

In [ ]:
!pip install pandas rdkit tqdm useful_rdkit_utils seaborn scikit-posthocs chembl_downloader

In [ ]:
import re

import chembl_downloader
import pandas as pd
import scikit_posthocs as sp
import seaborn as sns
import useful_rdkit_utils as uru
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.rdBase import BlockLogs
from tqdm.auto import tqdm

Enable progress bars in Pandas

In [ ]:
tqdm.pandas()

### 1. Download the ChEMBL database

[ChEMBL downloader](https://github.com/cthoyt/chembl-downloader) by Charles Tapley Hoyt

In [ ]:
path = chembl_downloader.download_extract_sqlite()

In [ ]:
path

PosixPath('/root/.data/chembl/35/data/chembl_35/chembl_35_sqlite/chembl_35.db')

Define SQL to extract drug data from ChEMBL. It would probably be useful to explain the query below.  We're joining three tables:
- molecule_dictionary - Table storing a non-redundant list of curated compounds for ChEMBL (includes preclinical compounds, drugs and clinical candidate drugs) and some associated attributes.
- compound_structures - Table storing various structure representations (e.g., Molfile, InChI) for each compound
- compound properties - Table storing calculated physicochemical properties for compounds, now calculated with RDKit and ChemAxon software

In [ ]:
sql="""SELECT
    cs.canonical_smiles,
    md.molregno,
    md.pref_name,
    act.standard_type,
    act.standard_value,
    act.standard_units,
    act.pchembl_value,
    t.pref_name AS target_name,
    t.target_type,
    t.chembl_id AS target_chembl_id
FROM
    molecule_dictionary md
JOIN
    compound_structures cs ON md.molregno = cs.molregno
JOIN
    activities act ON md.molregno = act.molregno
JOIN
    assays a ON act.assay_id = a.assay_id
JOIN
    target_dictionary t ON a.tid = t.tid
WHERE
    md.max_phase = 4
    AND md.molecule_type = 'Small molecule'
    AND act.standard_type IN ('IC50')"""


In [ ]:
df = chembl_downloader.query(sql)

In [ ]:
df

,canonical_smiles,molregno,pref_name,standard_type,standard_value,standard_units,pchembl_value,target_name,target_type,target_chembl_id
0,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,27307,ZIDOVUDINE,IC50,4.0,nM,8.40,ADMET,ADMET,CHEMBL612558
1,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,27307,ZIDOVUDINE,IC50,150.0,nM,6.82,ADMET,ADMET,CHEMBL612558
2,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,27307,ZIDOVUDINE,IC50,6.0,nM,8.22,ADMET,ADMET,CHEMBL612558
3,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,27307,ZIDOVUDINE,IC50,4.0,nM,8.40,ADMET,ADMET,CHEMBL612558
4,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,27307,ZIDOVUDINE,IC50,NaN,nM,NaN,Human immunodeficiency virus type 1 reverse tr...,SINGLE PROTEIN,CHEMBL247
...,...,...,...,...,...,...,...,...,...,...
202921,[2H]C([2H])([2H])NC(=O)c1nnc(NC(=O)C2CC2)cc1Nc...,2342179,DEUCRAVACITINIB,IC50,592.0,nM,6.23,Tyrosine-protein kinase JAK3,SINGLE PROTEIN,CHEMBL2148
202922,[2H]C([2H])([2H])NC(=O)c1nnc(NC(=O)C2CC2)cc1Nc...,2342179,DEUCRAVACITINIB,IC50,1900.0,nM,5.72,Tyrosine-protein kinase JAK3,SINGLE PROTEIN,CHEMBL2148
202923,[2H]C([2H])([2H])NC(=O)c1nnc(NC(=O)C2CC2)cc1Nc...,2342179,DEUCRAVACITINIB,IC50,10000.0,nM,NaN,Tyrosine-protein kinase TYK2,SINGLE PROTEIN,CHEMBL3553
202924,[2H]C([2H])([2H])NC(=O)c1nnc(NC(=O)C2CC2)cc1Nc...,2342179,DEUCRAVACITINIB,IC50,0.2,nM,9.70,Tyrosine-protein kinase TYK2,SINGLE PROTEIN,CHEMBL3553


In [ ]:
df.to_csv('chembl.csv', index=False)
from google.colab import files
files.download('chembl.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>